<a href="https://colab.research.google.com/github/YCPNG/data-course-sample/blob/main/S2_A2_Rule_based_RecSys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Code

## 基礎建設

In [161]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [162]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2021-12-26 09:39:31--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv.2’

All_Beauty.csv.2    100%[===================>]  14.78M  41.3MB/s    in 0.4s    

2021-12-26 09:39:32 (41.3 MB/s) - ‘All_Beauty.csv.2’ saved [15499476/15499476]

--2021-12-26 09:39:32--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz.2’

meta_All_Beauty.jso 100%[===================>]   9.85M  32.3MB/s    in 0.3s   

In [163]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [164]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [165]:
metadata.shape

(32892, 19)

In [166]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


In [167]:
ratings.shape

(371345, 4)

## 資料整理

In [168]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')
ratings.head()

,asin,reviewerID,overall,unixReviewTime,DATE
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000,2015-02-19
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800,2014-12-18
2,0143026860,A1572GUYS7DGSR,4.0,1407628800,2014-08-10
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000,2013-03-11
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200,2011-12-25


In [169]:
rank = metadata['rank']
rank_str = rank.str.split(expand = True)
rank_str = rank_str[0]
rank_num = pd.to_numeric(rank_str.str.replace(',',''))
rank_num

0        2938573.0
1         872854.0
2         956696.0
3        1870258.0
4          67701.0
           ...    
32887    2145325.0
32888    1639713.0
32889     207410.0
32890     965673.0
32891     478123.0
Name: 0, Length: 32892, dtype: float64

In [170]:
metadata['rank']=rank_num
metadata = metadata[['asin', 'brand', 'title', 'price', 'rank', 'description','also_buy', 'also_view']]
metadata.head()

,asin,brand,title,price,rank,description,also_buy,also_view
0,6546546450,idea village,Loud 'N Clear&trade; Personal Sound Amplifier,,2938573.0,[Loud 'N Clear Personal Sound Amplifier allows...,[],[]
1,7178680776,,No7 Lift &amp; Luminate Triple Action Serum 50...,$44.99,872854.0,[No7 Lift & Luminate Triple Action Serum 50ml ...,"[B01E7LCSL6, B008X5RVME]",[]
2,7250468162,No7,No7 Stay Perfect Foundation Cool Vanilla by No7,$28.76,956696.0,[No7 Stay Perfect Foundation now stays perfect...,[],"[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]"
3,7367905066,,Wella Koleston Perfect Hair Colour 44/44 Mediu...,,1870258.0,[],[B0041PBXX8],[]
4,7414204790,Pirmal Healthcare,Lacto Calamine Skin Balance Oil control 120 ml...,$12.15,67701.0,[Lacto Calamine Skin Balance Daily Nourishing ...,[],"[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4..."


In [171]:
metadata.sort_values(by=['rank'])

,asin,brand,title,price,rank,description,also_buy,also_view
8540,B00BXPVIT8,Braun,"Braun Clean &amp; Renew Refill Cartridges CCR,...",$21.44,35.0,"[With Braun Clean & Renew refills, you can kee...",[],"[B07L9PW8R3, B000KND8IS, B0786ZZQK6, B071KCNQT..."
6268,B006IB5T4W,Aquaphor,"Aquaphor Healing Ointment,Advanced Therapy Ski...",$11.29,65.0,"[Trusted by dermatologists, pediatricians and ...",[],"[B005UEB96K, B07HDDRBX8, B00E3H3260, B00005339..."
12028,B00HYEPVG8,Astra,Astra Platinum Double Edge Safety Razor Blades...,$9.59,245.0,[ASTRA stainless double edge blades fit all do...,[],"[B00DQ2W7UK, B00PKHIDRA, B07MB4LZC4, B07CGMTVM..."
3606,B001QY8QXM,Astra,Astra Platinum Double Edge Safety Razor Blades...,$9.30,280.0,[ASTRA stainless double edge blades fit all do...,[],"[B00DQ2W7UK, B00PKHIDRA, B07CGMTVM4, B07MB4LZC..."
481,B00005JS5C,Panasonic,Panasonic Bikini Shaper and Trimmer for Women ...,$18.49,457.0,"[Perfect for sensitive skin types, this bikini...",[],"[B00TYSQXIC, B0018A32XS, B000G0GW6K, B01IX3M3J..."
...,...,...,...,...,...,...,...,...
32324,B01GR35K9E,,Rise Shave Gel 2 oz. White (Pack of 12) by Rise,,NaN,[],[],[]
32335,B01GR3GB10,,Restor Style Ultra-soft Memory Foam Bath Pillo...,$45.01,NaN,[],[],[]
32648,B01H5IWC2S,7th Heaven,Strawberry Souffle by 7th Heaven,$8.86,NaN,[Souffle Mask;Non-Drying Moisurising & Soothin...,[],[]
32669,B01H66TI9E,,Le Angelique Hair Brush - Small Detangler Hair...,,NaN,[],[B01H67AB3A],[B01H67AB3A]


## 資料切分

In [172]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

In [ ]:
ratings_trainings

In [ ]:
users

## 產生推薦

In [ ]:
# 1. Recommend Top 10 products from training data period (before 2018-09-01)
def recommender(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    ratings_trainings = training_data
    
    asin = ratings_trainings.groupby(by=['asin']).size().sort_values(ascending=False)
    asin = asin.index.tolist()
    top_10_asin = asin[:10]

    recommendations = {user:top_10_asin for user in users}

    return recommendations


ratings_by_user_top10 = recommender(ratings_trainings, users)
ratings_by_user_top10

In [ ]:
# 2. Recommend top 10 & 5.0 rate products from training data period (before 2018-09-01)
def recommender(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}

    ratings_trainings = training_data

    ratings_trainings_5 = ratings_trainings[ratings_trainings['overall']==5.0]
    ratings_trainings_5 = ratings_trainings_5.groupby(by=['asin']).size().sort_values(ascending=False).index.tolist()[:10]

    recommendations = {user : ratings_trainings_5 for user in users}

    return recommendations
ratings_by_user_5 = recommender(ratings_trainings, users)
ratings_by_user_5

In [ ]:
# 3. Recommend top 10 products in 6 months
def recommender(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    ratings_trainings = training_data

    ratings_testings_6m = ratings[
    (ratings['DATE'] >= '2018-03-01') & 
    (ratings['DATE'] <= '2018-09-01')
    ]
    ratings_testings_6m_lst = ratings_testings_6m.groupby(by=['asin']).size().sort_values(ascending=False).index.tolist()[:10]

    recommendations = {user : ratings_testings_6m_lst for user in users}

    return recommendations
ratings_by_user_6m = recommender(ratings_trainings, users)
ratings_by_user_6m

## 結果評估

In [177]:
# 1. 評估推薦top10
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

evaluate(ratings_testings_by_user, ratings_by_user_top10)

0.08305084745762711

In [178]:
# 2. 評估推薦top10&評價5.0
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))
    score = total / len(ratings_testings)
    return score
evaluate(ratings_testings_by_user, ratings_by_user_5)

0.08305084745762711

In [179]:
# 3. 評估推薦6個月內的top10
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))
    score = total / len(ratings_testings)
    return score
evaluate(ratings_testings_by_user, ratings_by_user_6m)

0.09661016949152543